In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np

In [4]:
customer_info_df = pd.read_csv('/content/drive/MyDrive/Upay/Customer_Info.csv')
transaction_history_df = pd.read_csv('/content/drive/MyDrive/Upay/Transaction_History.csv')

FileNotFoundError: ignored

In [ ]:
customer_info_df.head()

,Name,Wallet No,Address,Gender,Balance,Age,Occupation,Loan Approval
0,Abdullah Al Mamun,1712345678,Urban,Male,29300.00,29,Business,Eligible
1,Ayesha Kabir,1887654321,Rural,Female,100.00,35,Student,UC
2,Jamal Ahmed,1913579246,Urban,Male,38250.00,38,Business,Eligible
3,Selina Begum,1624681012,Urban,Female,57547.30,39,Doctor,Eligible
4,Rahim Miah,1597531864,Urban,Male,2475.19,24,Student,Eligible


In [ ]:
transaction_history_df.head()

,Wallet No,DateTime,Transaction Type,Amount,Transaction ID,Balance
0,1712345678,1/1/2023 12:00,Received Money,1000.0,#01PJMOKOT4H,11000.0
1,1712345678,1/1/2023 12:00,Send Money,200.0,#01987R8SJFU,10800.0
2,1712345678,1/1/2023 12:00,Cash Out,500.0,#01NSY9KFFTO,10300.0
3,1712345678,2/5/2023 13:30,Make Payment,100.0,#011SYZ1UTZF,10200.0
4,1712345678,2/5/2023 13:30,Donation,50.0,#01U085ZYJMO,10150.0


In [1]:
# Define a function to calculate the coefficient of variation
def coefficient_of_variation(data):
    if len(data) < 2:
        return 0
    mean_balance = data[' Balance'].mean()
    std_balance = data[' Balance'].std()
    return (std_balance / mean_balance) * 100

In [2]:
# Calculate the coefficient of variation for each wallet's balances
avg_balances = {}
cv_df = transaction_history_df.groupby('Wallet No').apply(coefficient_of_variation).reset_index(name='Balance Deviation (CV)')


NameError: ignored

In [ ]:
cv_df

,Wallet No,Balance Deviation (CV)
0,1375318942,55.348391
1,1475316942,103.074102
2,1486429753,31.475020
3,1553197286,61.798848
4,1554311669,53.264535
5,1597531864,41.175715
6,1624681012,47.554104
7,1712345678,32.051120
8,1798765432,55.165339
9,1864281097,77.899895


In [ ]:
# Define the frequency ranges and transaction types dynamically
frequency_ranges = {
    'Send Money': {
        'Send Money_1': (10, 500),
        'Send Money_2': (501, 3000),
        'Send Money_3': (3001, 25000)
    },
    'Received Money': {
        'Received Money_1': (10, 500),
        'Received Money_2': (501, 3000),
        'Received Money_3': (3001, 25000)
    },
    'Cash In': {
        'Cash In_1': (10, 500),
        'Cash In_2': (501, 3000),
        'Cash In_3': (3001, 30000)
    },
    'Cash Out': {
        'Cash Out_1': (50, 500),
        'Cash Out_2': (501, 3000),
        'Cash Out_3': (3001, 25000)
    },
    'Make Payment': {
        'Make Payment_1': (1, 500),
        'Make Payment_2': (501, 3000),
        'Make Payment_3': (3001, 300000)
    },

    'Pay Bill': {
      'Pay Bill_1': (1, 2000),
      'Pay Bill_2': (2001, 300000)
    },

    'Add Money': {
      'Add Money_1': (50, 500),
      'Add Money_2': (501, 3000),
      'Add Money_3': (3001, 50000)
    },

    'Fund Transfer': {
      'Fund Transfer_1': (10, 500),
      'Fund Transfer_2': (501, 3000),
      'Fund Transfer_3': (3001, 50000)
    },

    'Request Money': {
      'Request Money': (10, 25000)
    },

    'Remittance': {
      'Remittance_1': (1, 10000),
      'Remittance_2': (10001, 125000)
    },

    'Donation': {
      'Donation_1': (10, 1500),
      'Donation_2': (1501, 300000)
    },

    'Others': {
      'Others_1': (1, 1500),
      'Others_2': (1501, 3000),
      'Others_3': (3001, 300000)
    },

}

In [ ]:
# Iterate through each transaction type and count transactions within the specified ranges
for transaction_type, ranges in frequency_ranges.items():
    if transaction_type == 'Others':
        # Check for specific transaction types and count them within the 'Others' ranges
        for range_name, (min_amount, max_amount) in ranges.items():
            for wallet_no, group in transaction_history_df.groupby('Wallet No'):
                transaction_count = ((group['Transaction Type'].isin(['Land Tax', 'E-Porcha', 'DNCC Holding Tax', 'E-Mutation'])) & (group['Amount'] >= min_amount) & (group['Amount'] <= max_amount)).sum()
                cv_df.loc[cv_df['Wallet No'] == wallet_no, range_name] = transaction_count
    else:
        # For other transaction types, count transactions based on the transaction type
        for range_name, (min_amount, max_amount) in ranges.items():
            for wallet_no, group in transaction_history_df.groupby('Wallet No'):
                transaction_count = ((group['Transaction Type'] == transaction_type) & (group['Amount'] >= min_amount) & (group['Amount'] <= max_amount)).sum()
                cv_df.loc[cv_df['Wallet No'] == wallet_no, range_name] = transaction_count

In [ ]:
# Convert the frequency columns to integer format
for column in cv_df.columns[2:]:
    cv_df[column] = cv_df[column].astype(int)

In [ ]:
cv_df

,Wallet No,Balance Deviation (CV),Send Money_1,Send Money_2,Send Money_3,Received Money_1,Received Money_2,Received Money_3,Cash In_1,Cash In_2,...,Fund Transfer_2,Fund Transfer_3,Request Money,Remittance_1,Remittance_2,Donation_1,Donation_2,Others_1,Others_2,Others_3
0,1375318942,55.348391,0,0,3,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1475316942,103.074102,0,7,4,2,0,0,0,2,...,0,0,1,0,0,0,1,0,1,1
2,1486429753,31.475020,0,20,4,17,14,15,2,5,...,2,0,5,0,0,6,0,0,0,0
3,1553197286,61.798848,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1554311669,53.264535,13,1,0,10,1,0,20,0,...,0,0,5,0,0,1,0,0,0,0
5,1597531864,41.175715,0,10,0,6,10,0,1,6,...,0,0,2,0,0,1,0,0,0,0
6,1624681012,47.554104,0,3,6,0,5,12,0,6,...,0,0,0,0,1,0,0,0,0,2
7,1712345678,32.051120,9,0,0,4,7,0,0,4,...,3,0,4,0,0,4,0,1,0,0
8,1798765432,55.165339,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9,1864281097,77.899895,0,1,1,0,1,2,0,0,...,1,0,2,0,0,0,0,0,0,1


In [ ]:
New_df = pd.merge(customer_info_df, cv_df, on='Wallet No', how='inner')

In [ ]:
New_df

,Name,Wallet No,Address,Gender,Balance,Age,Occupation,Loan Approval,Balance Deviation (CV),Send Money_1,...,Fund Transfer_2,Fund Transfer_3,Request Money,Remittance_1,Remittance_2,Donation_1,Donation_2,Others_1,Others_2,Others_3
0,Abdullah Al Mamun,1712345678,Urban,Male,29300.00,29,Business,Eligible,32.051120,9,...,3,0,4,0,0,4,0,1,0,0
1,Ayesha Kabir,1887654321,Rural,Female,100.00,35,Student,UC,63.098873,4,...,0,0,7,1,0,1,0,0,0,0
2,Jamal Ahmed,1913579246,Urban,Male,38250.00,38,Business,Eligible,39.525087,1,...,7,4,0,0,0,8,0,0,0,2
3,Selina Begum,1624681012,Urban,Female,57547.30,39,Doctor,Eligible,47.554104,0,...,0,0,0,0,1,0,0,0,0,2
4,Rahim Miah,1597531864,Urban,Male,2475.19,24,Student,Eligible,41.175715,0,...,0,0,2,0,0,1,0,0,0,0
5,Khadija Khatun,1486429753,Rural,Female,25090.47,42,Business,UC,31.475020,0,...,2,0,5,0,0,6,0,0,0,0
6,Ali Hossain,1375318942,Urban,Male,5880.18,36,Private Service,UC,55.348391,0,...,0,0,0,0,0,0,0,0,0,0
7,Fatima Akter,1864281097,Urban,Female,759.44,37,Lawyer,Not Eligible,77.899895,0,...,1,0,2,0,0,0,0,0,0,1
8,Karim Ahmed,1553197286,Urban,Male,2316.69,26,Farmer,Not Eligible,61.798848,0,...,0,0,0,0,0,0,0,0,0,0
9,Sultana Begum,1798765432,Rural,Female,4891.91,36,House Wife,Not Eligible,55.165339,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
New_df = New_df.drop('Name', axis=1)
New_df = New_df.drop('Gender', axis=1)
New_df

,Wallet No,Address,Balance,Age,Occupation,Loan Approval,Balance Deviation (CV),Send Money_1,Send Money_2,Send Money_3,...,Fund Transfer_2,Fund Transfer_3,Request Money,Remittance_1,Remittance_2,Donation_1,Donation_2,Others_1,Others_2,Others_3
0,1712345678,Urban,29300.00,29,Business,Eligible,32.051120,9,0,0,...,3,0,4,0,0,4,0,1,0,0
1,1887654321,Rural,100.00,35,Student,UC,63.098873,4,0,1,...,0,0,7,1,0,1,0,0,0,0
2,1913579246,Urban,38250.00,38,Business,Eligible,39.525087,1,8,4,...,7,4,0,0,0,8,0,0,0,2
3,1624681012,Urban,57547.30,39,Doctor,Eligible,47.554104,0,3,6,...,0,0,0,0,1,0,0,0,0,2
4,1597531864,Urban,2475.19,24,Student,Eligible,41.175715,0,10,0,...,0,0,2,0,0,1,0,0,0,0
5,1486429753,Rural,25090.47,42,Business,UC,31.475020,0,20,4,...,2,0,5,0,0,6,0,0,0,0
6,1375318942,Urban,5880.18,36,Private Service,UC,55.348391,0,0,3,...,0,0,0,0,0,0,0,0,0,0
7,1864281097,Urban,759.44,37,Lawyer,Not Eligible,77.899895,0,1,1,...,1,0,2,0,0,0,0,0,0,1
8,1553197286,Urban,2316.69,26,Farmer,Not Eligible,61.798848,0,0,2,...,0,0,0,0,0,0,0,0,0,0
9,1798765432,Rural,4891.91,36,House Wife,Not Eligible,55.165339,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
New_df.to_csv('/content/drive/MyDrive/Upay/New_df.csv', index = False)